<article class="day-desc"><h2>--- Day 1: Secret Entrance ---</h2><p>The Elves have good news and bad news.</p>
<p>The good news is that they've discovered <a href="https://en.wikipedia.org/wiki/Project_management" target="_blank">project management</a>! This has given them the tools they need to prevent their usual Christmas emergency. For example, they now know that the North Pole decorations need to be finished soon so that other critical tasks can start on time.</p>
<p>The bad news is that they've realized they have a <em>different</em> emergency: according to their resource planning, none of them have any time left to decorate the North Pole!</p>
<p>To save Christmas, the Elves need <em>you</em> to <em>finish decorating the North Pole by December 12th</em>.</p>
<p>Collect stars by solving puzzles.  Two puzzles will be made available on each day; the second puzzle is unlocked when you complete the first.  Each puzzle grants <em class="star">one star</em>. Good luck!</p>
<p>You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the <em>password</em> seems to have been changed, so you can't get in. A document taped to the wall helpfully explains:</p>
<p>"Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination."</p>
<p>The safe has a dial with only an arrow on it; around the dial are the numbers <code>0</code> through <code>99</code> in order. As you turn the dial, it makes a small <em>click</em> noise as it reaches each number.</p>
<p>The attached document (your puzzle input) contains a sequence of <em>rotations</em>, one per line, which tell you how to open the safe. A rotation starts with an <code>L</code> or <code>R</code> which indicates whether the rotation should be to the <em>left</em> (toward lower numbers) or to the <em>right</em> (toward higher numbers). Then, the rotation has a <em>distance</em> value which indicates how many clicks the dial should be rotated in that direction.</p>
<p>So, if the dial were pointing at <code>11</code>, a rotation of <code>R8</code> would cause the dial to point at <code>19</code>. After that, a rotation of <code>L19</code> would cause it to point at <code>0</code>.</p>
<p>Because the dial is a circle, turning the dial <em>left from <code>0</code></em> one click makes it point at <code>99</code>. Similarly, turning the dial <em>right from <code>99</code></em> one click makes it point at <code>0</code>.</p>
<p>So, if the dial were pointing at <code>5</code>, a rotation of <code>L10</code> would cause it to point at <code>95</code>. After that, a rotation of <code>R5</code> could cause it to point at <code>0</code>.</p>
<p>The dial starts by pointing at <code>50</code>.</p>
<p>You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is <em>the number of times the dial is left pointing at <code>0</code> after any rotation in the sequence</em>.</p>
<p>For example, suppose the attached document contained the following rotations:</p>
<pre><code>L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
</code></pre>
<p>Following these rotations would cause the dial to move as follows:</p>
<ul>
<li>The dial starts by pointing at <code>50</code>.</li>
<li>The dial is rotated <code>L68</code> to point at <code>82</code>.</li>
<li>The dial is rotated <code>L30</code> to point at <code>52</code>.</li>
<li>The dial is rotated <code>R48</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L5</code> to point at <code>95</code>.</li>
<li>The dial is rotated <code>R60</code> to point at <code>55</code>.</li>
<li>The dial is rotated <code>L55</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L1</code> to point at <code>99</code>.</li>
<li>The dial is rotated <code>L99</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>R14</code> to point at <code>14</code>.</li>
<li>The dial is rotated <code>L82</code> to point at <code>32</code>.</li>
</ul>
<p>Because the dial points at <code>0</code> a total of three times during this process, the password in this example is <code><em>3</em></code>.</p>
<p>Analyze the rotations in your attached document. <em>What's the actual password to open the door?</em></p>
</article>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from aocd import get_data

puzzle_input = get_data(day=1, year=2025)
puzzle_input[:20]

'R24\nR30\nL38\nL26\nL22\n'

In [3]:
sample_input = """L68
L30
R48
L5
R60
L55
L1
L99
R14
L82"""

sample_input

'L68\nL30\nR48\nL5\nR60\nL55\nL1\nL99\nR14\nL82'

In [4]:
from dataclasses import dataclass
from typing import Literal, Self, cast


@dataclass(frozen=True, slots=True)
class Rotation:
    """Represents a single dial rotation instruction."""

    direction: Literal["L", "R"]
    distance: int

    def __post_init__(self) -> None:
        if self.direction not in {"L", "R"}:
            raise ValueError("direction must be 'L' or 'R'")
        if self.distance < 0:
            raise ValueError("distance must be non-negative")

    @classmethod
    def from_string(cls, value: str) -> Self:
        """Parse compact rotation text like 'R30'."""
        direction_char = value[0]
        if direction_char not in {"L", "R"}:
            raise ValueError("direction must be 'L' or 'R'")
        direction = cast(Literal["L", "R"], direction_char)
        distance = int(value[1:])
        if distance < 0:
            raise ValueError("distance must be non-negative")
        return cls(direction=direction, distance=distance)


Rotation.from_string("R30")

Rotation(direction='R', distance=30)

In [5]:
@dataclass(frozen=True, slots=True)
class Instructions:
    """Collection of rotations parsed from a text block."""

    rotations: list[Rotation]

    @classmethod
    def from_string(cls, value: str) -> Self:
        """Parse newline-separated rotations into a structured list."""
        rotations = [
            Rotation.from_string(line.strip())
            for line in value.splitlines()
            if line.strip()
        ]
        return cls(rotations=rotations)


Instructions.from_string(sample_input)

Instructions(rotations=[Rotation(direction='L', distance=68), Rotation(direction='L', distance=30), Rotation(direction='R', distance=48), Rotation(direction='L', distance=5), Rotation(direction='R', distance=60), Rotation(direction='L', distance=55), Rotation(direction='L', distance=1), Rotation(direction='L', distance=99), Rotation(direction='R', distance=14), Rotation(direction='L', distance=82)])

In [6]:
@dataclass(frozen=True, slots=True)
class Dial:
    """Circular dial that tracks the current position and applies rotations."""

    value: int

    def __post_init__(self) -> None:
        if not 0 <= self.value <= 99:
            raise ValueError("value must be between 0 and 99 inclusive")

    def rotate(self, rotation: Rotation) -> Self:
        """Return a new dial after applying the rotation."""
        steps = rotation.distance % 100
        if rotation.direction == "L":
            new_value = (self.value - steps) % 100
        else:
            new_value = (self.value + steps) % 100
        return type(self)(new_value)


Dial(50).rotate(Rotation.from_string("R52"))

Dial(value=2)

In [7]:
from itertools import accumulate


def run_rotations(instructions: Instructions, start_value: int = 50) -> list[Dial]:
    """Return dial states including the starting position and after each rotation."""
    start_dial = Dial(start_value)
    return list(
        accumulate(
            instructions.rotations,
            lambda dial, rotation: dial.rotate(rotation),
            initial=start_dial,
        )
    )


run_rotations(
    instructions=Instructions.from_string(sample_input),
    start_value=50,
)

[Dial(value=50),
 Dial(value=82),
 Dial(value=52),
 Dial(value=0),
 Dial(value=95),
 Dial(value=55),
 Dial(value=0),
 Dial(value=99),
 Dial(value=0),
 Dial(value=14),
 Dial(value=32)]

In [8]:
def count_zero_positions(dials: list[Dial]) -> int:
    """Count how many dial states land exactly on zero."""
    return sum(dial.value == 0 for dial in dials)


count_zero_positions(dials=[Dial(0), Dial(10), Dial(0), Dial(25)])

2

In [9]:
sample_instructions = Instructions.from_string(sample_input)
sample_dials = run_rotations(sample_instructions)
sample_part1 = count_zero_positions(sample_dials)
assert sample_part1 == 3

In [10]:
puzzle_instructions = Instructions.from_string(puzzle_input)
puzzle_dials = run_rotations(puzzle_instructions)
part1 = count_zero_positions(puzzle_dials)
part1

1177

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think.</p>
<p>As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow:</p>
<p>"Due to newer security protocols, please use <em>password method <span title="You should have seen the chaos when the Elves overflowed their 32-bit password method counter.">0x434C49434B</span></em> until further notice."</p>
<p>You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times <em>any click</em> causes the dial to point at <code>0</code>, regardless of whether it happens during a rotation or at the end of one.</p>
<p>Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations:</p>
<ul>
<li>The dial starts by pointing at <code>50</code>.</li>
<li>The dial is rotated <code>L68</code> to point at <code>82</code>; during this rotation, it points at <code>0</code> <em>once</em>.</li>
<li>The dial is rotated <code>L30</code> to point at <code>52</code>.</li>
<li>The dial is rotated <code>R48</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L5</code> to point at <code>95</code>.</li>
<li>The dial is rotated <code>R60</code> to point at <code>55</code>; during this rotation, it points at <code>0</code> <em>once</em>.</li>
<li>The dial is rotated <code>L55</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L1</code> to point at <code>99</code>.</li>
<li>The dial is rotated <code>L99</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>R14</code> to point at <code>14</code>.</li>
<li>The dial is rotated <code>L82</code> to point at <code>32</code>; during this rotation, it points at <code>0</code> <em>once</em>.</li>
</ul>
<p>In this example, the dial points at <code>0</code> three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be <code><em>6</em></code>.</p>
<p>Be careful: if the dial were pointing at <code>50</code>, a single rotation like <code>R1000</code> would cause the dial to point at <code>0</code> ten times before returning back to <code>50</code>!</p>
<p>Using password method 0x434C49434B, <em>what is the password to open the door?</em></p>
</article>

In [ ]:
def zero_hits_for_rotation(start_dial: Dial, rotation: Rotation) -> int:
    """Return how many times zero is encountered during a single rotation."""
    steps = rotation.distance
    if steps == 0:
        return 0

    modulus = 100

    # Calculate how many clicks until the dial first reaches 0
    if rotation.direction == "R":
        # Right: distance from current position forward to 0
        # e.g., from 50 → 0 requires 50 clicks right
        first_hit = (modulus - start_dial.value) % modulus
    else:
        # Left: distance from current position backward to 0
        # e.g., from 50 → 0 requires 50 clicks left
        first_hit = start_dial.value % modulus

    # Special case: if already at 0, the next hit is a full rotation away
    if first_hit == 0:
        first_hit = modulus

    # If the rotation is too short to reach 0, no hits occur
    if steps < first_hit:
        return 0

    # Count: 1 for the first hit + additional full rotations (every 100 clicks)
    # e.g., 1000 clicks from 50: first hit at 50, then 9 more every 100 clicks = 10 total
    return 1 + (steps - first_hit) // modulus


zero_hits_for_rotation(
    start_dial=Dial(50),
    rotation=Rotation.from_string("R1000"),
)


10

In [12]:
def count_zero_clicks(instructions: Instructions, start_value: int = 50) -> int:
    """Count zero hits at every click using method 0x434C49434B."""
    dial_history = run_rotations(instructions, start_value)
    return sum(
        zero_hits_for_rotation(start_dial, rotation)
        for start_dial, rotation in zip(dial_history[:-1], instructions.rotations)
    )


sample_part2 = count_zero_clicks(sample_instructions)
assert sample_part2 == 6

In [13]:
part2 = count_zero_clicks(puzzle_instructions)
part2

6768